In [71]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers import Dense, Dropout, LSTM, Embedding, Reshape, Input, InputLayer, Concatenate, Lambda
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras import backend as K

In [130]:
n_samples = 5000
sample_inputs = []
sample_outputs = []
n_tokens = 61

patterns = [
    lambda i,l: list(range(i, i+l)),
    lambda i,l: list(range(i, i-l, -1))
]

for n in range(n_samples):
    sample_in = []
    sample_out = []
    for line_number,length in enumerate([5,7,5]):
        i = np.random.randint(10,50)
        #line = list(range(i, i+3))
        pattern = np.random.randint(len(patterns))
        line = patterns[pattern](i,length)
        '''if line_number == 0:
            sample_in.append([line[0]] + line)
        else:
            sample_in.append([0] + line)'''
        
        sample_in.append([line[0]] + line + [60]*(9-length))
            
        sample_out.append(line + [60]*(10-length))
        
        
        #sample.append(np.arange(i, i+3))
    #samples.append(sample)
    sample_inputs.append(sample_in)
    sample_outputs.append(sample_out)
    
X = np_utils.to_categorical(sample_inputs, num_classes=n_tokens)
Y = np_utils.to_categorical(sample_outputs, num_classes=n_tokens)

In [85]:
np.argmax([[0, 1, 2], [2, 1, 0]], axis=1) == 2

array([ True, False], dtype=bool)

In [131]:
latent_dim = 128

# Keras Model

inputs = [Input(shape=(None, n_tokens)) for i in range(3)]
lstms = [LSTM(latent_dim, return_state=True, return_sequences=True) for i in range(3)]

lstm_out, lstm_h, lstm_c = [None, None, None], [None, None, None], [None, None, None]
for i in range(3):
    if i > 0:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i], initial_state=[lstm_h[i-1], lstm_c[i-1]])
    else:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i])
        
denses = [Dense(n_tokens, activation='softmax') for i in range(3)]
outputs = [denses[i](lstm_out[i]) for i in range(3)]
#counter = Lambda(lambda x: K.sum(K.cast(K.equal(K.argmax(x, axis=2),60), 'float')))
#counter = Lambda(lambda x: K.sum(K.cast(K.equal(K.argmax(x, axis=1),60), K.floatx())))
counter = Lambda(lambda x: K.sum(K.cast(K.less(K.argmax(x, axis=2), 60), K.floatx()), axis=1, keepdims=True))
counts = [counter(i) for i in outputs]


model = Model(inputs, outputs + counts)
model.compile(optimizer='rmsprop', loss=['categorical_crossentropy']*3 + ['mean_squared_error']*3)
model.summary(line_length=200)

________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
input_85 (InputLayer)                                             (None, None, 61)                            0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
lstm_75 (LSTM)                                                    [(None, None, 128), (None, 128), (None, 128 97280                   input_85[0][0]                                                

In [133]:
r = model.predict([X[:1,0,:], X[:1,1,:], X[:1,2,:]])#[5]#.shape
r

[array([[[  2.93392119e-08,   2.34114790e-08,   2.53608654e-08,
            2.80961867e-08,   2.63066244e-08,   2.94444948e-08,
            1.12644202e-05,   4.98096870e-06,   3.27028329e-06,
            4.99554699e-05,   6.75999399e-05,   3.63395491e-04,
            1.11212015e-04,   1.29546082e-04,   4.74675180e-05,
            1.27538697e-05,   4.52261411e-05,   1.56820461e-04,
            3.34242563e-04,   4.67631646e-04,   4.06651379e-04,
            2.99630221e-04,   2.37682776e-04,   2.16334229e-04,
            2.34453983e-04,   2.33113824e-04,   5.98406114e-05,
            5.15583379e-05,   4.67923419e-05,   3.38946586e-04,
            3.51092545e-04,   2.99870939e-04,   1.04782041e-04,
            2.16267814e-04,   3.58597783e-04,   1.00059947e-02,
            6.12760223e-02,   8.24915767e-01,   8.08918774e-02,
            1.31218098e-02,   6.60303340e-04,   2.61030014e-04,
            3.65002401e-04,   1.07183272e-03,   7.34614092e-04,
            4.40338888e-04,   1.27107705

In [122]:
np.argmax(r[0][0], axis=1)

array([56, 56,  5, 42, 42, 34, 60, 60, 43, 43], dtype=int64)

In [126]:
np.ones(n_samples)*5

array([ 5.,  5.])

In [132]:
model.fit(
    [X[:,0,:], X[:,1,:], X[:,2,:]], 
    [
        Y[:,0,:], Y[:,1,:], Y[:,2,:], 
        np.ones(n_samples)*5, np.ones(n_samples)*7,  np.ones(n_samples)*5
    ],
    batch_size=64, epochs=20
)

Epoch 1/20
5000/5000 [==============================] - 5s 952us/step - loss: 44.7093 - dense_75_loss: 2.2489 - dense_76_loss: 2.8516 - dense_77_loss: 2.1212 - lambda_21_loss: 6.3334
Epoch 2/20
5000/5000 [==============================] - 2s 383us/step - loss: 15.6233 - dense_75_loss: 1.8622 - dense_76_loss: 2.4446 - dense_77_loss: 1.7491 - lambda_21_loss: 0.8148
Epoch 3/20
5000/5000 [==============================] - 2s 386us/step - loss: 6.5370 - dense_75_loss: 1.5389 - dense_76_loss: 1.9949 - dense_77_loss: 1.4063 - lambda_21_loss: 0.3658
Epoch 4/20
5000/5000 [==============================] - 2s 380us/step - loss: 4.4676 - dense_75_loss: 1.2840 - dense_76_loss: 1.5404 - dense_77_loss: 1.0945 - lambda_21_loss: 0.1160
Epoch 5/20
5000/5000 [==============================] - 2s 384us/step - loss: 3.2750 - dense_75_loss: 1.0627 - dense_76_loss: 1.1087 - dense_77_loss: 0.8094 - lambda_21_loss: 0.1020
Epoch 6/20
5000/5000 [==============================] - 2s 384us/step - loss: 2.4112 - d

In [134]:
generator_inputs = [Input(shape=(None, n_tokens), batch_shape=(1,None,n_tokens)) for i in range(3)]
generator_lstms = [LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True) for i in range(3)]
generator_lstm_out, generator_lstm_h, generator_lstm_c = [None, None, None], [None, None, None], [None, None, None]
generator_denses = [Dense(n_tokens, activation='softmax') for i in range(3)]

# Hook up line 0
generator_lstm_out[0], generator_lstm_h[0], generator_lstm_c[0] = generator_lstms[0](
    generator_inputs[0]
)
generator_outputs = [generator_denses[0](generator_lstm_out[0])]
generator_models = [Model(generator_inputs[0], [generator_outputs[0], generator_lstm_h[0], generator_lstm_c[0]])]

# Set weights for line 0
generator_lstms[0].set_weights(lstms[0].get_weights())
generator_denses[0].set_weights(denses[0].get_weights())

generator_models[0].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_88 (InputLayer)        (1, None, 61)             0         
_________________________________________________________________
lstm_78 (LSTM)               [(1, None, 128), (1, 128) 97280     
_________________________________________________________________
dense_78 (Dense)             (1, None, 61)             7869      
Total params: 105,149
Trainable params: 105,149
Non-trainable params: 0
_________________________________________________________________


In [135]:
generator_inputs_h = [None, Input(shape=(latent_dim,)), Input(shape=(latent_dim,))]
generator_inputs_c = [None, Input(shape=(latent_dim,)), Input(shape=(latent_dim,))]

#generator_inputs.append(Input(shape=(None, n_tokens), batch_shape=(1,None,n_tokens)))
#generator_lstms.append(LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True))


# Hook up lines 1 and 2
for i in [1, 2]:
    generator_lstm_out[i], generator_lstm_h[i], generator_lstm_c[i] = generator_lstms[i](
        generator_inputs[i], initial_state=[generator_inputs_h[i], generator_inputs_c[i]]
    )
    generator_outputs.append(generator_denses[i](generator_lstm_out[i]))
#generator_denses.append(Dense(n_tokens, activation='softmax'))

    generator_models.append(Model(
        [generator_inputs[i], generator_inputs_h[i], generator_inputs_c[i]],
        [generator_outputs[i], generator_lstm_h[i], generator_lstm_c[i]]
    ))

    generator_lstms[i].set_weights(lstms[i].get_weights())
    generator_denses[i].set_weights(denses[i].get_weights())

#generator_model_line1.summary()

In [49]:
#generator_model_line1.reset_states()
#generator_line1_lstm.reset_states()
#generator_lstms[1].reset_states()
line = [23]
#h = np.array([np.zeros(256)])
#c = np.array([np.zeros(256)])

def f():
    while line[-1] != 60:
        yield [np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), h, c]
        
while line[-1] != 60:
    character, h, c = generator_model_line1.predict_generator(f(), steps=1)
    line.append(np.argmax(character))
    
line

[23, 60]

In [61]:
#LINE 0
generator_models[0].reset_states()
line = [34]

def f():
    #while line[-1] != 60:
    while True:
        yield np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1))
        
#while line[-1] != 60:
while len(line) < 10:
    character, h, c = generator_models[0].predict_generator(f(), steps=1)
    line.append(np.argmax(character))
    
print(line)

# LINE 1
line = [49]

def f():
    while line[-1] != 60:
        yield [np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), h, c]
        
while line[-1] != 60:
    character, h, c = generator_models[1].predict_generator(f(), steps=1)
    line.append(np.argmax(character))
    
line

[34, 34, 33, 32, 31, 30, 60, 60, 60, 60]


[49, 49, 48, 47, 46, 45, 44, 43, 60]

In [141]:
h,c = None, None
generator_models[0].reset_states()
for i in range(3):
    line = [np.random.randint(10,50)]
    
    def f():
        while True:
            if i > 0:
                yield [np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), h, c]
            else:
                yield np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1))
        
    while len(line) < 10:
        character, h, c = generator_models[i].predict_generator(f(), steps=1)
        line.append(np.argmax(character))
        
    print(line[1:line.index(60)])

[26, 25, 24, 23, 22]
[42, 43, 44, 45, 46, 47, 48]
[12, 13, 14, 15, 16]


In [33]:
lstms[0].get_weights()

[array([[ 0.0588498 ,  0.07204304, -0.05792206, ..., -0.08177043,
          0.01025008,  0.08220639],
        [ 0.066603  ,  0.0180743 , -0.07614341, ...,  0.08674464,
         -0.09987819, -0.03462665],
        [ 0.00715201,  0.06240957,  0.09057698, ..., -0.07249868,
         -0.02441851,  0.04497065],
        ..., 
        [-0.05160541,  0.04952023, -0.08660023, ..., -0.08238563,
         -0.10212372, -0.04570872],
        [-0.04223401,  0.07268161,  0.0963813 , ..., -0.08047485,
          0.03274512,  0.01094449],
        [ 0.00469505,  0.03440818,  0.10505033, ..., -0.02963178,
          0.04027113, -0.03556556]], dtype=float32),
 array([[ 0.20038167,  0.1037483 , -0.12228944, ...,  0.34388208,
          0.37530544, -0.04092566],
        [ 0.39065197,  0.02621378,  0.12614504, ...,  0.33636111,
         -0.01913496,  0.1178768 ],
        [ 0.34746465,  0.12253693,  0.02178821, ...,  0.46020272,
          0.06336483, -0.06849611],
        ..., 
        [ 0.17965581,  0.33587414, -0

In [34]:
lstms[1].get_weights()

[array([[ 0.02373388,  0.04351699,  0.09366319, ..., -0.07357283,
          0.00148012, -0.07427015],
        [-0.02235256,  0.06999942, -0.05884961, ..., -0.05237408,
         -0.0445431 , -0.06093381],
        [ 0.04001717, -0.05314052, -0.07992874, ...,  0.06541879,
          0.0137012 ,  0.00431497],
        ..., 
        [ 0.01736447, -0.05911503, -0.00703574, ...,  0.00389478,
         -0.04105749,  0.07562602],
        [-0.04369491, -0.02797005, -0.03645102, ..., -0.03725483,
          0.07084001, -0.00418603],
        [-0.00543394, -0.04295987,  0.11130186, ..., -0.25775337,
         -0.07856624,  0.02195806]], dtype=float32),
 array([[ 0.16149174, -0.01383852, -0.04173021, ...,  0.16967967,
         -0.18976605,  0.06395357],
        [-0.04755745,  0.01412346,  0.00423123, ...,  0.18604733,
         -0.05554095,  0.12966727],
        [ 0.18584771, -0.02389204, -0.08125761, ...,  0.18993607,
         -0.00577669,  0.06357786],
        ..., 
        [ 0.06819594,  0.19305532,  0

In [36]:
generator_lstms[1].get_weights()

[array([[ 0.02373388,  0.04351699,  0.09366319, ..., -0.07357283,
          0.00148012, -0.07427015],
        [-0.02235256,  0.06999942, -0.05884961, ..., -0.05237408,
         -0.0445431 , -0.06093381],
        [ 0.04001717, -0.05314052, -0.07992874, ...,  0.06541879,
          0.0137012 ,  0.00431497],
        ..., 
        [ 0.01736447, -0.05911503, -0.00703574, ...,  0.00389478,
         -0.04105749,  0.07562602],
        [-0.04369491, -0.02797005, -0.03645102, ..., -0.03725483,
          0.07084001, -0.00418603],
        [-0.00543394, -0.04295987,  0.11130186, ..., -0.25775337,
         -0.07856624,  0.02195806]], dtype=float32),
 array([[ 0.16149174, -0.01383852, -0.04173021, ...,  0.16967967,
         -0.18976605,  0.06395357],
        [-0.04755745,  0.01412346,  0.00423123, ...,  0.18604733,
         -0.05554095,  0.12966727],
        [ 0.18584771, -0.02389204, -0.08125761, ...,  0.18993607,
         -0.00577669,  0.06357786],
        ..., 
        [ 0.06819594,  0.19305532,  0